In [4]:
from RT_prediction.RT_prediction import *
import matplotlib.pyplot as plt
import numpy as np

In [5]:
#define predictor object, n_jobs sets the number of parralel processes to use
RT_pred = RT_predictor(n_jobs=4)

#get training data from pHILIC. This is molecular descriptors and RTs
training_data = RT_pred.get_training_data_pHILIC_IROA()

#get descriptors that can be used for training data
goodCols_train = RT_pred.get_good_desc(training_data)

#these properties are computed by running the R script compute_molecular_properites.R. 
pred_props = pd.read_csv("../data/example_molecular_properties.csv")
goodCols_pred = RT_pred.get_good_desc(pred_props)
preds = list(set(goodCols_train).intersection(set(goodCols_pred)))
print(len(preds),preds)

#train model and print cv r2
nBest,mBest,max_depth = (100,51,200)
print(RT_pred.crossValPrediction(training_data[preds].to_numpy(),training_data["RT"].values,nBest,mBest,max_depth))
RT_pred.fit(training_data[preds].to_numpy(),training_data["RT"].values,nBest,mBest,max_depth)


Unnamed: 0 : non-real valued descriptor
Name : non-real valued descriptor
Unnamed: 0 : non-real valued descriptor
data[i, 1] : non-real valued descriptor
117 ['SPC.5', 'MDEC.12', 'ATSp1', 'SP.7', 'VC.3', 'VCH.5', 'C2SP3', 'SC.6', 'SCH.3', 'SPC.6', 'ATSp4', 'VCH.7', 'C2SP2', 'C1SP2', 'VAdjMat', 'ATSm5', 'fragC', 'Zagreb', 'SP.6', 'WPOL', 'MDEN.22', 'C3SP2', 'MDEC.22', 'VC.5', 'nHBDon', 'MW', 'SC.5', 'SCH.4', 'SC.3', 'VP.4', 'nAtomLAC', 'LipinskiFailures', 'nAromRings', 'C3SP3', 'ATSp5', 'ECCEN', 'VP.6', 'SPC.4', 'nRings3', 'VCH.4', 'nAcid', 'MDEO.12', 'VPC.4', 'MDEC.24', 'SCH.5', 'MDEN.23', 'ATSp2', 'nRingBlocks', 'WTPT.1', 'VP.3', 'nAtomP', 'topoShape', 'MDEO.11', 'SP.0', 'AMR', 'SP.1', 'nRings6', 'VC.4', 'nHBAcc', 'ALogp2', 'WTPT.3', 'VC.6', 'SCH.7', 'XLogP', 'ATSm3', 'PetitjeanNumber', 'SCH.6', 'TopoPSA', 'WTPT.5', 'SP.4', 'VPC.6', 'SP.2', 'nAromBond', 'VP.5', 'ATSm1', 'naAromAtom', 'MDEC.23', 'VP.0', 'MDEN.33', 'ALogP', 'VP.2', 'nRotB', 'nAtomLC', 'WPATH', 'MLogP', 'VP.7', 'MDEC.33'

In [7]:
#predict RTs
rt_predicted = RT_pred.predict(pred_props[preds].to_numpy())

#get cpd names
names = pred_props[pred_props.columns.values[-1]].values

#output RTs
pd.DataFrame.from_dict({n:{"name":n,"RT":rt} for n,rt in zip(names,rt_predicted)},orient="index").to_csv("../data/example_predicted_rts.csv",index=False)